In [ ]:
# import libraries
import numpy as np
import pandas as pd
from __future__ import division

In [ ]:
#1. Set your own stopping conditions for pre-prunning
'''
Here, I would like to use running time as as stoppping condition
To do so, I would keep track of the running time and stop the programme
once it exceeds the maximum limit
'''
def stop_6(current_split, max_split):
    
    if current_split>=max_split:
        print "Stop Condition 6: Exceed maximum number of splits"
        return True
    
    return False



#Add the the new stop condition to the tree

def ClassificationTree(samples, output, features, step, tree_depth, max_depth, min_number, min_infogain, split, max_split):
    '''This function is used to build a classification tree in a recursive way.
       Remember how you build a binary tree in the previous Java and Data Structure courses).
       
    Inputs:
    1) samples: Samples in the current tree node before making split on the feature (Pandas Dataframe)
    2) output: Name of the output column
    3) features: A list of feature names
    4) step: The current binary split step
    5) tree_depth: The depth of the current tree
    6) max_depth: Maximum depth this tree can grow
    7) min_number: Minimum number of node size
    8) min_infogain: Minimum information gain
    9）split: current number of split
    10) max_split: the maximum number of split
    
    Outputs:
    1) tree_nodes: Nested tree nodes, which are stored and shown in nested dictionary type    
    
    '''
    
    # If samples are empty, return None
    if samples is None or len(samples)==0:
        return None
           
    
    current_features = features # Current feature list
    labels = samples[output] # Output labels in the current tree node

    print "----------------------------------------------------------------------------"
    print "----------------------------------------------------------------------------"
    print "Step %s: Current tree depth is %s. Current tree node has %s data points" % (step, tree_depth, len(samples)) # Sample size
    
    # Verify whether stopping conditions 1-4,6 are satisfied. If satisfied, return a leaf_node
    if stop_1(labels) or stop_2(current_features) or stop_3(tree_depth, max_depth) or stop_4(samples, min_number) or stop_6(split, max_split):
        return {
                'label': majority_vote(labels),
                'left_tree': None,
                'right_tree': None,
                'best_feature': None          
            
                }
    
    # If pass stopping conditions 1-4 and 6, then do best splitting
    best_split = best_feature_split(samples, output, current_features)
    best_feature, best_infogain, best_left, best_right = best_split[0], best_split[1], best_split[2], best_split[3]
    
    # Verify whether stopping condition 5 is satisfied. If satisfied, return a leaf node
    if stop_5(best_infogain, min_infogain):
        return {
                'label': majority_vote(labels),
                'left_tree': None,
                'right_tree': None,
                'best_feature': None          
            
                } 
    
    # If pass stopping condition 5, then move on
    step += 1
    split += 1
    
    print "Step %s: Binary split on %s. Size of Left and Right tree is (%s, %s)" % \
          (step, best_feature, len(best_left) if best_left is not None else 0, len(best_right) if best_right is not None else 0)
    current_features.remove(best_feature) # Remove this feature if this feature is used for split
    
    # Do binary split on left tree and right tree in a recursive way
    left_split = ClassificationTree(best_left, output, current_features, step+1, tree_depth+1, max_depth, min_number, min_infogain, split, max_split) 
    right_split = ClassificationTree(best_right, output, current_features, step+1, tree_depth+1, max_depth, min_number, min_infogain, split, max_split) 
    
    return {
            'label': None,
            'left_tree': left_split,
            'right_tree': right_split,
            'best_feature': best_feature        
            
            }  


#Test the new build-tree function
features = list(dataset.columns[2:])
output = dataset.columns[1]
#tree_model = ClassificationTree(dataset_copy, output, features, step=0, tree_depth=0, max_depth=7, min_number=5, min_infogain=5e-4, split = 0, max_split = 5)
#tree_model
dataset.head(n = 10)

In [ ]:
# 2. Try to do post prunning
'''
  General Idea: 
  One common aproach of post-prunig aims to retain the decision tree but to replace some of its subtrees by leaf nodes,
thus converting a complete tree to a smaller pruned one which predicts the classification of unseen instances at 
least as accurately.
  we look for non-leaf nodes in the tree that has a descendant subtree of 1. Then we use prunning dataset to calculate
the misclassification rate at each node. For every non-leaf nodes in the tree with descendant subtree of 1, we calculate
and compare its misclassification rate with the weighted average of its child nodes. If it is smaller, we turn that node
into a leaf. Continue this process recursively until there are no nodes to prun.

'''

# Before we wirte the code for post prunning, we modify the original decision tree function and add a couple of more functions

#Modify the build tree function in the tutorial, so that it includenumber of instances in the training set
#that belongs to the leaf
def ClassificationTree(samples, output, features, step, tree_depth, max_depth, min_number, min_infogain):
    '''This function is used to build a classification tree in a recursive way.
       Remember how you build a binary tree in the previous C++ and Data Structure courses).
       
    Inputs:
    1) samples: Samples in the current tree node before making split on the feature (Pandas Dataframe)
    2) output: Name of the output column
    3) features: A list of feature names
    4) step: The current binary split step
    5) tree_depth: The depth of the current tree
    6) max_depth: Maximum depth this tree can grow
    7) min_number: Minimum number of node size
    8) min_infogain: Minimum information gain
    Outputs:
    1) tree_nodes: Nested tree nodes, which are stored and shown in nested dictionary type    
    
    '''
    if sample is None or len(samples) == 0:
        return None
    
    current_features = features # Current feature list
    labels = samples[output] # Output labels in the current tree node

    print "----------------------------------------------------------------------------"
    print "----------------------------------------------------------------------------"
    print "Step %s: Current tree depth is %s. Current tree node has %s data points" % (step, tree_depth,len(samples))
    
    # Verify whether stopping conditions 1-4 and 6 are satisfied. If satisfied, return a leaf_node
    if stop_1(labels) or stop_2(current_features) or stop_3(tree_depth, max_depth) or stop_4(samples, min_number):
        return {
                'label': majority_vote(labels),
                'left_tree': None,
                'right_tree': None,
                'best_feature': None,
                'number': len(labels)
                }
    # If pass stopping conditions 1-4, then do best splitting
    best_split = best_feature_split(samples, output, current_features)
    best_feature, best_infogain, best_left, best_right = (best_split[0], best_split[1], best_split[2], best_split[3])
    
    # Verify whether stopping condition 5 is satisfied. If satisfied, return a leaf node
    if stop_5(best_infogain, min_infogain):
        return {
                'label': majority_vote(labels),
                'left_tree': None,
                'right_tree': None,
                'best_feature': None,
                'number': len(labels)
            
                } 
    # If pass stopping condition 5, then move on
    step += 1
    print "Step %s: Binary split on %s. Size of Left and Right tree is (%s, %s)" % (step, best_feature, len(best_left), len(best_right))
    current_features.remove(best_feature) # Remove this feature if this feature is used for split
    
    # Do binary split on left tree and right tree in a recursive way
    left_split = ClassificationTree(best_left, output, current_features, step+1, tree_depth+1, max_depth, min_number, min_infogain)
    right_split = ClassificationTree(best_right, output, current_features, step+1, tree_depth+1, max_depth, min_number, min_infogain)
    
    return {
            'label': None,
            'left_tree': left_split,
            'right_tree': right_split,
            'best_feature': best_feature,
            'number': 0
            
            }  


def majority_vote(output_labels):
    
    '''This function is used to get predicted label based on "Majority Voting" criterion for the current leaf node.     
    Inputs:
    1) output_labels: Outputs (labels) in this leaf node, such as [1, 0, 0, 1, 1]
    
    Outputs:
    1) prediction: Predicted label for this leaf node (e.g., 0/-1, or 1)
    
    '''    
    # numpy array
    output_labels = np.array(output_labels)
    
    # Empty label
    if output_labels.size == 0:
        return None
    
    # Count output labels (0/-1 or 1)
    values = np.unique(output_labels)
    
    if len(values) == 1:
        return values[0]
    else:
        num0 = len(output_labels[output_labels == values[0]])
        num1 = len(output_labels[output_labels == values[1]])
        return values[1] if num1 >= num0 else values[0]  # Prediction based on "Majority Voting" criterion   
        

In [ ]:
'''
Post prunning Cotinued
'''
# Functon to find the deepest internal node

def deepest_node(tree,path):
    '''
    inputs:
    tree: the decision tree
    path: an empty list to contain the path later
    '''
    if tree['label'] and path:  #reach a leaf
        path.pop() #pop up the last move
        return path;
    
    #make a deep copy of the original past list
    left_path = list(path)
    right_path = list(path)
    
    left_path.append('left')
    right_path.append('right')
    
    #recursion
    left = deepest_node(tree['left_tree'],left_path)
    right = deepest_node(tree['right_tree'], right_path)
    
    #if the left tree is deeper
    if len(left)>=len(right):
        return left
    else:
        return right
    
    
    
#function to replace a node with a leaf by the majority of classification
def replace_deepest_node(tree):
    
    feature = dict();
    path = deepest_node(tree,[])
    for i in path:
        if i=='left':
            feature = tree['left_tree']
        else:
            feature = tree['right_tree']
    
    #turn the internal node to a leaf
    left = feature['left_tree']
    right = feature['right_tree']
    
    #find the number of instances of each leaf
    numberLeft = left['number']
    numberRight = right['number']
    
    #eliminate the original child leaf
    feature['left_tree'] = None
    feature['right_tree'] = None
    
    feature['number'] = numberLeft + numRight
    
    feature['label'] = left['label'] if numberLeft>=numberRight else right['label']
    
    return 
    
        
#The final prunning function
import copy

def post_prunning(tree, test_data,features,output):
    '''
    input:
    tree: the decision tree generated
    test_data: test data to measure the performance of the tree
    features: name of the features to be selected
    output: the name of the output column
    
    outputs:
    return the updated tree
    '''
    # if the root of tree is a leaf
    if tree['label']:
        return tree
    
    actual_labels = test_data[output]
    error_rate = MissClassificationRate(tree,test_data[features],actual_labels)
    
    temp_tree = copy.deepcopy(tree)
    
    replace_deepest_node(tree)
    
    new_error_rate = MissClassificatonRate(tree,test_data[features],actual_labels)
    
    if(error_rate<new_error_rate): //the error rate increases when do the prunning
        return temp_tree
    else:
        return post_prunning(tree,test_data,features,output)

In [ ]:
#3. Use Gini_index for binary splitting
def Gini_index(sample_labels):

    # sample is a list including numbers representing classification group.
    # e.g. [0,0,1,1,1]
    sample_labels = np.array(sample_labels)
    if sample_labels.size == 0:
        return 0
    class_values = np.unique(sample_labels)
    if class_values.size == 1:
        return 0 
    num1 = len(filter(lambda x: x == class_values[0],sample_labels))
    num2 = len(filter(lambda x: x == class_values[1],sample_labels))
    p1 = num1/(num1+num2)
    p2 = 1 - p1
    gini_index = 1-p1**2-p2**2
    return gini_index



In [ ]:
#4. Write a function to calculate missclassification rate
def MissClassificationRate(train_tree,table,actual_labels):
    predicted_label = []
    
    for i in range(table.shape(0)):
        new_sample = table.iloc(i)  # extract the row of table
        predicted_label.append(predict_label(new_sample,train_tree))
    
    if len(predicted_labels)!=len(actual_labels):
        print "Wrong prediction: number of elements mismatch"
        return 1;
    
    n = len(actual_labels)
    wrong = 0
    for i in range(n):
        if(predicted_labels[i] != actual_labels[i]):
            wrong+=1
            
    return wrong/n
    

#The predict_label function in the tutorial notes
def predict_label(new_sample, train_tree):   
    '''This function is used to predict the label of one new sample.
    Inputs:
    1) new_sample: A new sample, we would like to predict its label (Pandas DataFrame)
    2) train_tree: The classification tree we have just trained
    
    Outputs:
    1) predict_label: The predicted label for this new sample  
    
    '''
    
    # If move to the leaf node
    if train_tree['best_feature']== None:
        return train_tree['label']
    # If still stay at temporary node
    
    else:
        # Find the value of the best feature in the current node
        # If value is 0, then go to left tree
        # If value is 1, then go to right tree
        # Remember what your have learned in Data Structure course, about binary tree
        best_feature = train_tree['best_feature']
        return predict_label(new_sample, train_tree['left_tree']) if new_sample[best_feature]==0 else predict_label(new_sample, train_tree['right_tree'])
        
    
    
    


In [ ]:
#5.1 What if features are continuous? Expain in words what would happen
'''
For features that are continous, we need to use threshold values to split the variable
into several non-overlapping ranges and then treate it as a categorical variable. The cut
points should reflect real properties of the domain.

Methods to do this include 
A. local discretisation: 
    1. For each continuous attribute A
     a) Sort the instances into ascending numerical order.
     b) If there are n distinct values v1, v2, ..., vn, calculate the values of information gain 
     (or other measure) for each of the n−1 corresponding pseudo-attributes A < v2, A < v3, ..., A < vn.
     c) Find which of the n − 1 attribute values gives the largest value of information gain 
     (or optimises some other measure). If this is vi re- turn the pseudo-attribute A < vi, and the value of the 
     corresponding measure.
    2. Calculate the value of information gain (or other measure) for any categor- ical attributes.
    3. Select the attribute or pseudo-attribute with the largest value of informa- tion gain 
    (or which optimises some other measure).

B. global discretisation: ChiMerge algorithm

'''

In [ ]:
#5.2 What if output is continuous? Explain in words what would happen
'''
When output is continuous, still, we use threshold values to split the variable
into several non-overlapping ranges and then treate it as a categorical variable.
'''